In [1]:
from simpleai.search import SearchProblem, greedy

class KnapsackProblem(SearchProblem):
    def __init__(self, weights, values, capacity):
        self.weights = weights
        self.values = values
        self.capacity = capacity
        self.n = len(weights)
        super().__init__(initial_state=(0, tuple()))  # (used_capacity, items_taken as tuple)

    def actions(self, state):
        used_capacity, items_taken = state
        return [i for i in range(self.n)
                if i not in items_taken and used_capacity + self.weights[i] <= self.capacity]

    def result(self, state, action):
        used_capacity, items_taken = state
        new_items_taken = items_taken + (action,)  # dùng tuple thay vì list
        return (used_capacity + self.weights[action], new_items_taken)

    def is_goal(self, state):
        return not self.actions(state)

    def heuristic(self, state):
        _, items_taken = state
        return -sum(self.values[i] for i in items_taken)

# Dữ liệu
weights = [4, 3, 2]
values  = [100, 60, 50]
capacity = 5

# Giải bài toán
problem = KnapsackProblem(weights, values, capacity)
result = greedy(problem)

print("Các vật phẩm được chọn:", result.state[1])
print("Tổng giá trị đạt được:", sum(values[i] for i in result.state[1]))

Các vật phẩm được chọn: (0,)
Tổng giá trị đạt được: 100


In [2]:
#fractional knapsack
from simpleai.search import SearchProblem, greedy

class FractionalKnapsackProblem(SearchProblem):
    def __init__(self, weights, values, capacity):
        self.weights = weights
        self.values = values
        self.capacity = capacity
        self.n = len(weights)
        initial_state = (0.0, tuple([0.0] * self.n))  # (capacity_used, fractions_taken) -> state: (0.0, (0.0, 0.0, ... , 0.0))
        super().__init__(initial_state)

    def actions(self, state):
        used_capacity, taken = state
        actions = []
        for i in range(self.n):
            if taken[i] < 1.0 and used_capacity < self.capacity:
                actions.append(i)
        return actions

    def result(self, state, action):
        used_capacity, taken = state
        i = action

        remaining_capacity = self.capacity - used_capacity
        item_weight = self.weights[i]
        item_remaining_fraction = 1.0 - taken[i]

        max_can_take = min(1.0, remaining_capacity / item_weight, item_remaining_fraction)

        new_taken = list(taken)
        new_taken[i] += max_can_take

        return (used_capacity + item_weight * max_can_take, tuple(new_taken))

    def is_goal(self, state):
        used_capacity, taken = state
        return used_capacity >= self.capacity or all(f >= 1.0 for f in taken)

    def heuristic(self, state):
        used_capacity, taken = state
        remaining_capacity = self.capacity - used_capacity
        estimate = 0.0

        ratios = [(i, self.values[i] / self.weights[i]) for i in range(self.n) if taken[i] < 1.0]
        ratios.sort(key=lambda x: -x[1])  # sort by value/weight descending

        for i, ratio in ratios:
            can_take = min(self.weights[i] * (1.0 - taken[i]), remaining_capacity)
            estimate += ratio * can_take
            remaining_capacity -= can_take
            if remaining_capacity <= 0:
                break

        return -estimate  # GBFS muốn heuristic nhỏ nhất

    def value(self, state):
        _, taken = state
        return sum(self.values[i] * taken[i] for i in range(self.n))


# Dữ liệu ví dụ
weights = [2,1,3,2]
values = [12,10,20,15]
capacity = 5

# Khởi tạo và giải
problem = FractionalKnapsackProblem(weights, values, capacity)
result = greedy(problem, graph_search=True)

# In kết quả
print(f"Tổng giá trị tối đa: {problem.value(result.state):.2f}")
for i, frac in enumerate(result.state[1]):
    if frac > 0:
        print(f"Vật phẩm {i}: lấy {frac*100:.1f}% (trọng lượng {weights[i]}, giá trị {values[i]})")

Tổng giá trị tối đa: 35.33
Vật phẩm 0: lấy 100.0% (trọng lượng 2, giá trị 12)
Vật phẩm 1: lấy 100.0% (trọng lượng 1, giá trị 10)
Vật phẩm 2: lấy 66.7% (trọng lượng 3, giá trị 20)


In [3]:
import math

def bounded_knapsack_greedy(weights, values, quantities, capacity):
    n = len(weights)
    used_capacity = 0
    items_taken = [0] * n  # Số lượng từng vật được lấy

    while True:
        if used_capacity >= capacity:
            break

        # Tìm các hành động hợp lệ (chưa lấy hết số lượng)
        actions = [i for i in range(n) if items_taken[i] < quantities[i]]

        if not actions:
            break

        # Chọn vật có tỉ lệ giá trị/trọng lượng cao nhất
        best_action = max(actions, key=lambda i: values[i] / weights[i])

        remaining_capacity = capacity - used_capacity

        # Tính số lượng tối đa có thể lấy thêm vật này
        max_can_take = min(
            quantities[best_action] - items_taken[best_action],
            math.floor(remaining_capacity / weights[best_action])
        )

        # Nếu không thể lấy thêm vật nào thì dừng
        if max_can_take == 0:
            break

        # Cập nhật trạng thái
        items_taken[best_action] += max_can_take
        used_capacity += weights[best_action] * max_can_take

    return used_capacity, items_taken
weights = [2, 3, 5]
values = [50, 60, 140]
quantities = [3, 2, 1]
capacity = 10

used_capacity, items_taken = bounded_knapsack_greedy(weights, values, quantities, capacity)

print("Dung lượng đã sử dụng:", used_capacity)
print("Số lượng từng vật được chọn:", items_taken)

# Tính tổng giá trị
total_value = sum(values[i] * items_taken[i] for i in range(len(weights)))
print("Tổng giá trị đạt được:", total_value)


Dung lượng đã sử dụng: 9
Số lượng từng vật được chọn: [2, 0, 1]
Tổng giá trị đạt được: 240


In [1]:
import heapq
from datetime import datetime

place_names = [
    'Bảo tàng Quốc gia', 'Công viên Trung tâm', 'Di tích Lịch Sử',
    'Đồi Nghệ Thuật', 'Chợ đêm Sài Gòn', 'Hồ Thiên Nga',
    'Làng Văn Hóa', 'Vườn Thực Vật', 'Nhà hát Thành Phố'
]

places = [(2, 10), (1, 7), (4, 15), (3, 8), (1, 5),
          (3, 11), (2, 9), (3, 6), (1, 8)]  # (giờ, điểm)

def heuristic(index, remaining_time, selected_places):
    total_score = 0
    count = 0
    for t, s in sorted(selected_places[index:], key=lambda x: x[0]):
        if t <= remaining_time:
            total_score += s
            count += 1
            remaining_time -= t
    return total_score + count * 2

def gbfs_tour_knapsack(max_time, selected_places, selected_ids):
    steps = []
    pq = []
    heapq.heappush(pq, (-heuristic(0, max_time, selected_places), 0, 0, 0, []))
    best_score = 0
    best_tour = []
    while pq:
        h, cur_val, cur_time, idx, selected = heapq.heappop(pq)
        steps.append(f"Xét node {idx} | score: {cur_val} | thời gian: {cur_time} | chọn: {[place_names[i] for i in selected]}")
        if cur_val > best_score:
            best_score = cur_val
            best_tour = selected
        if idx >= len(selected_places):
            continue
        time_needed, score = selected_places[idx]
        place_id = selected_ids[idx]
        heapq.heappush(pq, (-heuristic(idx+1, max_time-cur_time, selected_places), cur_val, cur_time, idx+1, selected))
        if cur_time + time_needed <= max_time:
            heapq.heappush(pq, (-heuristic(idx+1, max_time-(cur_time+time_needed), selected_places),
                                cur_val+score, cur_time+time_needed, idx+1, selected + [place_id]))
    return best_tour, best_score, steps

# === INPUT ===
start_time = "09:00"
end_time = "17:00"
selected_indices = list(range(len(place_names)))  # chọn tất cả

# === XỬ LÝ ===
start = datetime.strptime(start_time, "%H:%M")
end = datetime.strptime(end_time, "%H:%M")
max_time = (end - start).seconds // 3600

sub_places = [places[i] for i in selected_indices]
tour, score, steps = gbfs_tour_knapsack(max_time, sub_places, selected_indices)

# === OUTPUT ===
print("🎯 Tổng điểm đạt được:", score)
print("🕐 Tổng thời gian sử dụng:", sum(places[i][0] for i in tour), "giờ")
print("📍 Số địa điểm tham quan:", len(tour))
print("🔗 Lộ trình tối ưu:")
for idx, i in enumerate(tour, 1):
    print(f"   {idx}. {place_names[i]} ({places[i][0]}h, {places[i][1]}đ)")
print("\n🧭 Các bước thuật toán GBFS:")
print("\n".join(steps))


🎯 Tổng điểm đạt được: 41
🕐 Tổng thời gian sử dụng: 8 giờ
📍 Số địa điểm tham quan: 5
🔗 Lộ trình tối ưu:
   1. Bảo tàng Quốc gia (2h, 10đ)
   2. Công viên Trung tâm (1h, 7đ)
   3. Chợ đêm Sài Gòn (1h, 5đ)
   4. Hồ Thiên Nga (3h, 11đ)
   5. Nhà hát Thành Phố (1h, 8đ)

🧭 Các bước thuật toán GBFS:
Xét node 0 | score: 0 | thời gian: 0 | chọn: []
Xét node 1 | score: 0 | thời gian: 0 | chọn: []
Xét node 2 | score: 0 | thời gian: 0 | chọn: []
Xét node 3 | score: 0 | thời gian: 0 | chọn: []
Xét node 4 | score: 0 | thời gian: 0 | chọn: []
Xét node 2 | score: 7 | thời gian: 1 | chọn: ['Công viên Trung tâm']
Xét node 3 | score: 7 | thời gian: 1 | chọn: ['Công viên Trung tâm']
Xét node 4 | score: 7 | thời gian: 1 | chọn: ['Công viên Trung tâm']
Xét node 1 | score: 10 | thời gian: 2 | chọn: ['Bảo tàng Quốc gia']
Xét node 5 | score: 0 | thời gian: 0 | chọn: []
Xét node 5 | score: 5 | thời gian: 1 | chọn: ['Chợ đêm Sài Gòn']
Xét node 5 | score: 7 | thời gian: 1 | chọn: ['Công viên Trung tâm']
Xét node 